In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install nilearn

In [ ]:
import tensorflow as tf
import keras_preprocessing
import keras as k
import keras.backend as K
import pandas as pd
import nibabel as nb
import numpy as np
import os
from mlxtend.preprocessing import minmax_scaling
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Input
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import UpSampling3D
from keras.layers.convolutional import MaxPooling3D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv3D, Conv2D
from keras.optimizers import RMSprop, Adam, Adadelta, Adagrad, SGD
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import r2_score

In [ ]:
from nilearn.plotting import view_img, glass_brain, plot_anat, plot_epi

In [ ]:
import nilearn

In [ ]:
cd drive/My\ Drive

In [ ]:
pwd

In [ ]:
def bbox(img):

    r = np.any(img, axis=(1, 2))
    c = np.any(img, axis=(0, 2))
    z = np.any(img, axis=(0, 1))

    rmin, rmax = np.where(r)[0][[0, -1]]
    cmin, cmax = np.where(c)[0][[0, -1]]
    zmin, zmax = np.where(z)[0][[0, -1]]

    return rmin, rmax, cmin, cmax, zmin, zmax

In [ ]:
def load_train():
    x_train = []
    y_train1 = []
    y_train2 = []
    y_train3 = []

    heights = pd.read_csv('Pre-operative_TCGA-GBM_Segmentations/Pre-operative_TCGA-GBM_Segmentations/TCGA_GBM_radiomicFeatures.csv')
    
    for i in range(102):
      img_path1 = os.path.join('Pre-operative_TCGA-GBM_Segmentations', 'Pre-operative_TCGA-GBM_Segmentations', str(heights['ID'][i]), str(heights['ID'][i])+'_'+str(heights['Date'][i]) + '_t1.nii.gz')
      img_path2 = os.path.join('Pre-operative_TCGA-GBM_Segmentations', 'Pre-operative_TCGA-GBM_Segmentations', str(heights['ID'][i]), str(heights['ID'][i])+'_'+str(heights['Date'][i]) + '_t1Gd.nii.gz')
      img_path3 = os.path.join('Pre-operative_TCGA-GBM_Segmentations', 'Pre-operative_TCGA-GBM_Segmentations', str(heights['ID'][i]), str(heights['ID'][i])+'_'+str(heights['Date'][i]) + '_t2.nii.gz')
      img_path4 = os.path.join('Pre-operative_TCGA-GBM_Segmentations', 'Pre-operative_TCGA-GBM_Segmentations', str(heights['ID'][i]), str(heights['ID'][i])+'_'+str(heights['Date'][i]) + '_flair.nii.gz')
      img_path5 = os.path.join('Pre-operative_TCGA-GBM_Segmentations', 'Pre-operative_TCGA-GBM_Segmentations', str(heights['ID'][i]), str(heights['ID'][i])+'_'+str(heights['Date'][i]) + '_GlistrBoost_ManuallyCorrected.nii.gz')
      im5 = niCopy of Untitled9.learn.image.math_img('img > 1', img=img_path5)
      im1 = nb.load(img_path1)
      im2 = nb.load(img_path2)
      im3 = nb.load(img_path3)
      im4 = nb.load(img_path4)
      #im5 = nb.load(img_path5)
      img1 = np.logical_and(im1, im5)
      img2 = np.logical_and(im2, im5)
      img3 = np.logical_and(im3, im5)
      img4 = np.logical_and(im4, im5)
      img5 = img1.get_data()
      img6 = img2.get_data()
      img7 = img3.get_data()
      img8 = img4.get_data()
      rmin1, rmax1, cmin1, cmax1, zmin1, zmax1 = bbox(img5)
      rmin2, rmax2, cmin2, cmax2, zmin2, zmax2 = bbox(img6)
      rmin3, rmax3, cmin3, cmax3, zmin3, zmax3 = bbox(img7)
      rmin4, rmax4, cmin4, cmax4, zmin4, zmax4 = bbox(img8)
      i1 = img5[rmin1:rmin1+94,cmin1:cmin1+85,zmin1:zmin1+72]
      i2 = img6[rmin2:rmin2+94,cmin2:cmin2+85,zmin2:zmin3+72]
      i3 = img7[rmin3:rmin3+94,cmin3:cmin3+85,zmin3:zmin3+72]
      i4 = img8[rmin4:rmin4+94,cmin4:cmin4+85,zmin4:zmin4+72]
      #img5 = nb.load(img_path5).get_data()
      #plot_anat(img5)
      #image = np.stack((i1,i2,i3,i4),axis=-1)
      image = np.dstack((i1,i2,i3,i4))
      print(image.shape)
      #let1 = nilearn.image.new_img_like(im1,i1 )
      #plot_anat(let1)
      x_train.append(image)
      y_train1.append(heights['TGM_p1'][i])
      y_train2.append(heights['TGM_dw'][i])
      y_train3.append(heights['TGM_T_1'][i])

      print(i)
    return x_train, y_train1,y_train2,y_train3

In [ ]:
z,y,e,f=load_train()

In [ ]:
z=np.asarray(z)

In [ ]:
z.shape

In [ ]:
y=np.asarray(y)
e=np.asarray(e)
f=np.asarray(f)

In [ ]:
seed = 52
np.random.seed(seed)

In [ ]:
split = train_test_split(f, e, y, z, test_size=0.2, random_state=seed)
(Y3_train, Y3_test, Y2_train, Y2_test, Y1_train, Y1_test, X_train, X_test) = split

In [ ]:
X_train.shape

In [ ]:
def nrmse(actual,predicted):
  return (k.backend.sqrt(k.backend.mean(k.backend.square(abs(actual-predicted)))))/(k.backend.mean(actual))

In [ ]:
y

In [ ]:
pwd

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(z, y):
  # create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
	model.fit(z[train], y[train], epochs=150, batch_size=10, verbose=0)
	# evaluate the model
	scores = model.evaluate(z[test], y[test], verbose=0)
	print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
	cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

In [ ]:
acc_per_fold = []
loss_per_fold = []
loss_plot = []
kfold = KFold(n_splits= 10, shuffle=True,random_state = seed)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(z, y):

  # Define the model architecture
  inputs = Input(shape=(94, 85, 288))

  # convolutional layer

  aa = Conv2D(16, kernel_size=(1,1),strides=(1,1),activation='relu')(inputs)
  aa = Conv2D(16, kernel_size=(3,3),strides=(1,1),activation='relu')(aa)
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  aa = Conv2D(32, kernel_size=(3,3),strides=(1,1),activation='relu')(aa)
  aa = Conv2D(32, kernel_size=(3,3),strides=(1,1),activation='relu')(aa)
  aa = MaxPooling2D(pool_size=(2,2))(aa)
  
  
  ff = Flatten()(aa)

  # hidden layer
  hh = Dense(160,activation='relu')(ff)
  hh = Dropout(0.2)(hh)
  # output layer
  output1 = Dense(1, activation='linear', name='TGM_T_1')(hh)



  model = Model(inputs = inputs , outputs = [output1])

  # compiling the sequential model
  opt = Adagrad()
  model.compile(loss='mse', optimizer=opt, metrics=[nrmse])



  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  nabh = model.fit(z[train], y[train],batch_size=8, epochs=100,verbose=0)
  loss_train = nabh.history['nrmse']
  
  epochs = range(0,100)
  sns.set() # Use seaborn's default style to make attractive graphs
  plt.rcParams['figure.dpi'] = 340 
  plt.plot(epochs, loss_train, 'g', label='Training accuracy')
  
  plt.title('loss vs epoch')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

  # Generate generalization metrics
  scores = model.evaluate(z[test], y[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}%')
  acc_per_fold.append(scores[1])
  loss_per_fold.append(scores[0])
  loss_plot.append(nabh.history['nrmse'])
  # Increase fold number
  fold_no = fold_no + 1


# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Mape: {acc_per_fold[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Mape: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  print('------------------------------------------------------------------------')

In [ ]:
m = model.predict(X_test)
m

In [ ]:
Y1_test

In [ ]:
plt.plot(range(0,10),'r')
plt.scatter(m,Y1_test)

In [ ]:
r = r2_score(Y1_test,m)

In [ ]:
r

In [ ]:
scipy.stats.spearmanr(m,Y1_test).correlation

In [ ]:
plt.plot(loss_plot[6])

plt.xlim(0, 100)

In [ ]:
import scipy

aa = Conv2D(16, kernel_size=(1,1),strides=(1,1),activation='relu')(inputs)
  aa = Conv2D(16, kernel_size=(3,3),strides=(1,1),activation='relu')(aa)
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  aa = Conv2D(32, kernel_size=(3,3),strides=(1,1),activation='relu')(aa)
  aa = Conv2D(32, kernel_size=(3,3),strides=(1,1),activation='relu')(aa)
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  ff = Flatten()(aa)

  # hidden layer
  hh = Dense(110,activation='relu')(ff)
  hh = Dropout(0.2)(hh)

  # output layer
  output1 = Dense(1, activation='linear', name='TGM_T_1')(hh)





  batch size = 8


  TGM_T_1

 aa = Conv2D(16, kernel_size=(1,1),strides=(1,1),activation='relu')(inputs)
  aa = Conv2D(16, kernel_size=(1,1),strides=(1,1),activation='relu')(aa)
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  aa = Conv2D(32, kernel_size=(1,1),strides=(1,1),activation='relu')(aa)
  aa = Conv2D(32, kernel_size=(1,1),strides=(1,1),activation='relu')(aa)
  aa = MaxPooling2D(pool_size=(2,2))(aa)
  
  
  ff = Flatten()(aa)

  # hidden layer
  hh = Dense(180,activation='relu')(ff)
  hh = Dropout(0.2)(hh)
  # output layer
  output1 = Dense(1, activation='linear', name='TGM_T_1')(hh)



  model = Model(inputs = inputs , outputs = [output1])

  # compiling the sequential model
  opt = Adam()
  model.compile(loss='mse', optimizer=opt, metrics=[nrmse])



  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  nabh = model.fit(z[train], en[train],batch_size=8, epochs=100,verbose=0)



In [ ]:
en = e/1e-9

In [ ]:
def nmse(actual,predicted):
  return k.backend.mean(k.backend.square(abs(actual-predicted)))/1e-4

In [ ]:
acc_per_fold1 = []
loss_per_fold1 = []
loss_plot1 = []
kfold = KFold(n_splits= 10, shuffle=True,random_state = seed)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(z, en):

  # Define the model architecture
  inputs = Input(shape=(94, 85, 288))

  # convolutional layer

  aa = Conv2D(8, kernel_size=(1,1),strides=(1,1),activation='relu')(inputs) 
  aa = Conv2D(8, kernel_size=(1,1),strides=(1,1),activation='relu')(aa) 
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  aa = Conv2D(16, kernel_size=(1,1),strides=(1,1),activation='relu')(aa) 
  aa = Conv2D(16, kernel_size=(1,1),strides=(1,1),activation='relu')(aa) 
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  ff = Flatten()(aa)

  #hidden layer
  hh = Dense(140,activation='relu')(ff) 
  hh = Dropout(0.2)(hh)

  #output layer
  output1 = Dense(1, activation='linear', name='TGM_T_1')(hh)

  model1 = Model(inputs = inputs , outputs = [output1])

  #compiling the sequential model
  opt = Adam() 
  model1.compile(loss='mse', optimizer=opt, metrics=[nrmse])

  #Generate a print
  print('------------------------------------------------------------------------') 
  print(f'Training for fold {fold_no} ...')

  nabh1 = model1.fit(z[train], en[train],batch_size=8, epochs=150,verbose=0)

  loss_train1 = (nabh1.history['nrmse'])

  epochs1 = range(0,150)
  sns.set() # Use seaborn's default style to make attractive graphs
  plt.rcParams['figure.dpi'] = 340 
  plt.plot(epochs1, loss_train1, 'g', label=f'Fold {fold_no}')

  plt.title('loss vs epoch')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

  # Generate generalization metrics
  scores = model1.evaluate(z[test], en[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}%')
  acc_per_fold1.append(scores[1])
  loss_per_fold1.append(scores[0])
  loss_plot1.append(nabh1.history['nrmse'])
  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold1[i]} - Mape: {acc_per_fold1[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Mape: {np.mean(acc_per_fold1)} (+- {np.std(acc_per_fold1)})')
  print(f'> Loss: {np.mean(loss_per_fold1)}')
  print('------------------------------------------------------------------------')

In [ ]:
m1 = model1.predict(X_test)
m1

In [ ]:
Y2_test/1e-9

In [ ]:
plt.plot(range(0,800),'r')
plt.scatter(m1,Y2_test/1e-9)

In [ ]:
r1 = r2_score(Y2_test/1e-9,m1)

In [ ]:
r1

In [ ]:
import scipy

In [ ]:
scipy.stats.spearmanr(m1,Y2_test/1e-9).correlation

In [ ]:
plt.plot(loss_plot1[6])

plt.xlim(0, 150)

In [ ]:
acc_per_fold2 = []
loss_per_fold2 = []
loss_plot2 = []
kfold = KFold(n_splits= 10, shuffle=True,random_state = seed)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(z, f):

  # Define the model architecture
  inputs = Input(shape=(94, 85, 288))

  aa = Conv2D(16, kernel_size=(3,3),strides=(1,1),activation='relu')(inputs) 
  aa = Conv2D(16, kernel_size=(3,3),strides=(1,1),activation='relu')(aa) 
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  aa = Conv2D(32, kernel_size=(3,3),strides=(1,1),activation='relu')(aa) 
  aa = Conv2D(32, kernel_size=(3,3),strides=(1,1),activation='relu')(aa) 
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  ff = Flatten()(aa)

  #hidden layer
  hh = Dense(160,activation='relu')(ff) 
  hh = Dropout(0.2)(hh)

  #output layer
  output1 = Dense(1, activation='linear', name='TGM_T_1')(hh)



  model2 = Model(inputs = inputs , outputs = [output1])

  # compiling the sequential model
  opt = Adagrad()
  model2.compile(loss='mse', optimizer=opt, metrics=[nrmse])



  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  nabh2 = model2.fit(z[train], f[train],batch_size=8, epochs=150,verbose=0)

  loss_train2 = (nabh2.history['nrmse'])

  epochs2 = range(0,150)
  sns.set() # Use seaborn's default style to make attractive graphs
  plt.rcParams['figure.dpi'] = 340 
  plt.plot(epochs2, loss_train2, 'g', label=f'Fold {fold_no}')

  plt.title('loss vs epoch')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

  # Generate generalization metrics
  scores = model2.evaluate(z[test], f[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}%')
  acc_per_fold2.append(scores[1])
  loss_per_fold2.append(scores[0])
  loss_plot2.append(nabh2.history['nrmse'])
  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold2)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold2[i]} - Mape: {acc_per_fold2[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Mape: {np.mean(acc_per_fold2)} (+- {np.std(acc_per_fold2)})')
  print(f'> Loss: {np.mean(loss_per_fold2)}')
  print('------------------------------------------------------------------------')

In [ ]:
m2 = model2.predict(X_test)
m2

In [ ]:
Y3_test

In [ ]:
plt.plot(range(0,750),'r')
plt.scatter(m2,Y3_test)

In [ ]:
r2 = r2_score(Y3_test,m2)

In [ ]:
r2

In [ ]:
import scipy

In [ ]:
scipy.stats.spearmanr(m2,Y3_test).correlation

In [ ]:
plt.plot(loss_plot2[2])

plt.xlim(0, 150)

In [ ]:
plt.rcParams['figure.facecolor'] = 'cyan'
plt.plot(epochs, loss_plot[6], 'k-.', label=f'Mp', lw=2.6)
plt.plot(epochs1, loss_plot1[6], 'r--', label=f'Dw', lw=3)
plt.plot(epochs2, loss_plot2[2], 'g', label=f'T', lw = 3.2)
plt.title('loss(nrmse) vs epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc_per_fold3 = []
loss_per_fold3 = []
Mp_fold = []
kfold = KFold(n_splits= 10, shuffle=True,random_state = seed)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(z, y):

  # Define the model architecture
  inputs = Input(shape=(94, 85, 288))

  # convolutional layer

  aa = Conv2D(16, kernel_size=(1,1),strides=(1,1),activation='relu')(inputs)
  aa = Conv2D(16, kernel_size=(3,3),strides=(1,1),activation='relu')(aa)
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  aa = Conv2D(32, kernel_size=(3,3),strides=(1,1),activation='relu')(aa)
  aa = Conv2D(32, kernel_size=(3,3),strides=(1,1),activation='relu')(aa)
  aa = MaxPooling2D(pool_size=(2,2))(aa)
  
  
  ff = Flatten()(aa)

  # hidden layer
  hh = Dense(160,activation='relu')(ff)
  hh = Dropout(0.2)(hh)
  # output layer
  output1 = Dense(1, activation='linear', name='TGM_T_1')(hh)



  model3 = Model(inputs = inputs , outputs = [output1])

  # compiling the sequential model
  opt = Adagrad()
  model3.compile(loss='mse', optimizer=opt, metrics=[nrmse])



  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  nabh3 = model3.fit(z[train], y[train],batch_size=8, epochs=100,verbose=0)
  loss_train = nabh3.history['nrmse']
  
  epochs = range(0,100)
  sns.set() # Use seaborn's default style to make attractive graphs
  plt.rcParams['figure.dpi'] = 340 
  

  # Generate generalization metrics
  scores = model3.evaluate(z[test], y[test], verbose=0)
  print(f'Score for fold {fold_no}: {model3.metrics_names[0]} of {scores[0]}; {model3.metrics_names[1]} of {scores[1]}%')
  acc_per_fold3.append(scores[1])
  loss_per_fold3.append(scores[0])
  
  Mp_fold.append(f'Fold {fold_no}')
  m3 = model3.predict(X_train)
  r3 = r2_score(Y1_train,m3)
  s3 = scipy.stats.spearmanr(m3,Y1_train).correlation
  
  if fold_no==0:
    m3s = pd.DataFrame(np.column_stack([Mp_fold, r3, s3]), columns=[f'Fold_Mp', f'pear_corr', f'spear_corr'], index = False)
    m3s.to_csv('corre_Mp.csv')
  else:
    m3s = pd.DataFrame(np.column_stack([Mp_fold, r3, s3])) 
    m3s.to_csv('corre_Mp.csv', mode='a')
  # Increase fold number
  Mp_fold.pop()
  fold_no = fold_no + 1
 

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold3)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold3[i]} - Mape: {acc_per_fold3[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Mape: {np.mean(acc_per_fold3)} (+- {np.std(acc_per_fold3)})')
  print(f'> Loss: {np.mean(loss_per_fold3)}')
  print('------------------------------------------------------------------------')

In [ ]:
acc_per_fold4 = []
loss_per_fold4 = []
Dw_plot = []
kfold = KFold(n_splits= 10, shuffle=True,random_state = seed)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(z, en):

  # Define the model architecture
  inputs = Input(shape=(94, 85, 288))

  # convolutional layer

  aa = Conv2D(8, kernel_size=(1,1),strides=(1,1),activation='relu')(inputs) 
  aa = Conv2D(8, kernel_size=(1,1),strides=(1,1),activation='relu')(aa) 
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  aa = Conv2D(16, kernel_size=(1,1),strides=(1,1),activation='relu')(aa) 
  aa = Conv2D(16, kernel_size=(1,1),strides=(1,1),activation='relu')(aa) 
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  ff = Flatten()(aa)

  #hidden layer
  hh = Dense(140,activation='relu')(ff) 
  hh = Dropout(0.2)(hh)

  #output layer
  output1 = Dense(1, activation='linear', name='TGM_T_1')(hh)

  model4 = Model(inputs = inputs , outputs = [output1])

  #compiling the sequential model
  opt = Adam() 
  model4.compile(loss='mse', optimizer=opt, metrics=[nrmse])

  #Generate a print
  print('------------------------------------------------------------------------') 
  print(f'Training for fold {fold_no} ...')

  nabh4 = model4.fit(z[train], en[train],batch_size=8, epochs=150,verbose=0)

  # Generate generalization metrics
  scores = model4.evaluate(z[test], en[test], verbose=0)
  print(f'Score for fold {fold_no}: {model4.metrics_names[0]} of {scores[0]}; {model4.metrics_names[1]} of {scores[1]}%')
  acc_per_fold4.append(scores[1])
  loss_per_fold4.append(scores[0])

  Dw_plot.append(f'Fold {fold_no}')
  m4 = model4.predict(X_train)
  r4 = r2_score(Y2_train/1e-9,m4)
  s4 = scipy.stats.spearmanr(m4,Y2_train/1e-9).correlation
  
  if fold_no==0:
    m4s = pd.DataFrame(np.column_stack([Dw_plot, r4, s4]), columns=[f'Fold_Dw', f'pear_corr', f'spear_corr'], index = False)
    m4s.to_csv('corre_Dw.csv')
  else:
    m4s = pd.DataFrame(np.column_stack([Dw_plot, r4, s4])) 
    m4s.to_csv('corre_Dw.csv', mode='a')
  # Increase fold number
  Dw_plot.pop()
  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold4)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold4[i]} - Mape: {acc_per_fold4[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Mape: {np.mean(acc_per_fold4)} (+- {np.std(acc_per_fold4)})')
  print(f'> Loss: {np.mean(loss_per_fold4)}')
  print('------------------------------------------------------------------------')

In [ ]:
acc_per_fold5 = []
loss_per_fold5 = []
T_fold = []
kfold = KFold(n_splits= 10, shuffle=True,random_state = seed)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(z, f):

  # Define the model architecture
  inputs = Input(shape=(94, 85, 288))

  aa = Conv2D(16, kernel_size=(3,3),strides=(1,1),activation='relu')(inputs) 
  aa = Conv2D(16, kernel_size=(3,3),strides=(1,1),activation='relu')(aa) 
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  aa = Conv2D(32, kernel_size=(3,3),strides=(1,1),activation='relu')(aa) 
  aa = Conv2D(32, kernel_size=(3,3),strides=(1,1),activation='relu')(aa) 
  aa = MaxPooling2D(pool_size=(2,2))(aa)

  ff = Flatten()(aa)

  #hidden layer
  hh = Dense(160,activation='relu')(ff) 
  hh = Dropout(0.2)(hh)

  #output layer
  output1 = Dense(1, activation='linear', name='TGM_T_1')(hh)



  model5 = Model(inputs = inputs , outputs = [output1])

  # compiling the sequential model
  opt = Adagrad()
  model5.compile(loss='mse', optimizer=opt, metrics=[nrmse])



  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  nabh5 = model5.fit(z[train], f[train],batch_size=8, epochs=150,verbose=0)

  # Generate generalization metrics
  scores = model5.evaluate(z[test], f[test], verbose=0)
  print(f'Score for fold {fold_no}: {model5.metrics_names[0]} of {scores[0]}; {model5.metrics_names[1]} of {scores[1]}%')
  acc_per_fold5.append(scores[1])
  loss_per_fold5.append(scores[0])
  
  T_fold.append(f'Fold {fold_no}')
  m5 = model5.predict(X_train)
  r5 = r2_score(Y3_train,m5)
  s5 = scipy.stats.spearmanr(m5,Y3_train).correlation
  
  if fold_no==0:
    m5s = pd.DataFrame(np.column_stack([T_fold, r5, s5]), columns=[f'Fold_T', f'pear_corr', f'spear_corr'], index = False)
    m5s.to_csv('corre_T.csv')
  else:
    m5s = pd.DataFrame(np.column_stack([T_fold, r5, s5])) 
    m5s.to_csv('corre_T.csv', mode='a')
  # Increase fold number
  T_fold.pop()
  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold5)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold5[i]} - Mape: {acc_per_fold5[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Mape: {np.mean(acc_per_fold5)} (+- {np.std(acc_per_fold5)})')
  print(f'> Loss: {np.mean(loss_per_fold5)}')
  print('------------------------------------------------------------------------')

In [ ]:
df = pd.read_csv('CORRELATION_VALUES.csv')

plt.rcParams['figure.facecolor'] = 'cyan'

plt.title('Correlation Distribution for 10 folds')

sns.boxplot( x=[df["Pearson_T"],df["Spearman_T"], df["Pearson_Dw"],df["Spearman_Dw"], df["Pearson_Mp"],df["Spearman_Mp"]],y=['Pearson_T','Spearman_T','Pearson_Dw','Spearman_Dw','Pearson_Mp','Spearman_Mp'], width = 0.5 ).set(
    xlabel='Correlation Values')
#plt.ylim(0.970, 1.00)
plt.show()